In [42]:
import mar
import os

from mar.wrappers.wrappersoperation import *

from astropy.table import Table
from astropy.io import fits

In [82]:
tuple(list((6, 30.0, 5.0, 7.0, 10.0, 35.0)))

(6, 30.0, 5.0, 7.0, 10.0, 35.0)

In [83]:
def createVignetCatalog(image, output):
    conf = {
        "CATALOG_NAME": output,

        "WEIGHT_TYPE": "NONE",

        "BACK_FILTERSIZE": 5,
        "BACK_SIZE": 512,
        "DETECT_THRESH": 10.0,
        "ANALYSIS_THRESH": 10.0,
        "DETECT_MINAREA": 10,

        "CATALOG_TYPE": 'FITS_LDAC',
        "PHOT_AUTOPARAMS": '2.5,3.5',
        "CHECKIMAGE_TYPE": "NONE",
        "PHOT_FLUXFRAC": "0.5",

        "FILTER_NAME": os.path.join(mar.__path__[0], 'config/sextractor/gauss_3.0_5x5.conv'), 
        "FILTER": "Y",
        "PHOT_APERTURES": ("%.2f, %.2f, %.2f, %.2f, %.2f, %.2f" % (6, 30.0, 5.0, 7.0, 10.0, 35.0))

    }

    numapers = 6
    vignetsize = [51, 51]

    params = [
                "NUMBER",
                "ALPHA_J2000",
                "DELTA_J2000",
                "VIGNET({0},{1})".format(vignetsize[0], vignetsize[1]),
                "X_IMAGE", "Y_IMAGE",
                "FLUX_AUTO", "FLUXERR_AUTO",
                "MAG_AUTO", "MAGERR_AUTO",
                "FLUX_APER({0})".format(numapers),
                "FLUXERR_APER({0})".format(numapers),
                "MAG_APER({0})".format(numapers),
                "MAGERR_APER({0})".format(numapers),
                "FWHM_IMAGE", "ISOAREA_IMAGE",
                "FLUX_RADIUS", "FLUX_MAX",
                "ELLIPTICITY", "ELONGATION",
                "SNR_WIN",
                "FLAGS",
                "FLAGS_WEIGHT",
                "CLASS_STAR",
                "XWIN_IMAGE", "YWIN_IMAGE"
    ]

    from mar.wrappers.sextractor import SExtr

    inst = SExtr(image=image, folder="./outputs", addconf=conf)
    inst.params = params
    inst.run()

In [84]:
createVignetCatalog("/home/gustavo/Downloads/_media_files_HYDRA-0023_R_swp", "outputs/final.fits")



> 
----- SExtractor 2.25.2 started on 2023-02-07 at 14:50:27 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing catalog
> Looking for _media_files_HYDRA-0023_R_swp
----- Measuring from: _media_files_HYDRA-0023_R_swp
      "HYDRA_0023" / no ext. header / 11000x11000 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:  512
> Setting up background map at line: 1024
> Setting up background map at line: 1536
> Setting up background map at line: 2048
> Setting up background map at line: 2560
> Setting up background map at line: 3072
> Setting up background map at line: 3584
> Setting up background map at line: 4096
> Setting up background map at line: 4608
> Setting up background map at line: 5120
> Setting up background map at line: 5632
> Setting up background map at line: 6144
> Setting up background map at line: 6656
> Setting up 

In [85]:
def filterCatalogToPSFex(tablepath):
	t = Table.read(tablepath, hdu=2)
	f = fits.open(tablepath, mode='update')

	selection = (
				(t["FLAGS"] == 0) &
				(t["MAGERR_AUTO"] < 0.01) &
				(t["ELLIPTICITY"] < 0.2) &
				(t["ISOAREA_IMAGE"] > 40) &
				(t["SNR_WIN"] < 1000)
			)

	f[2].data = t[selection].as_array()
	f.flush()

In [89]:
filterCatalogToPSFex("outputs/final.fits")

In [90]:
len(Table.read("outputs/final.fits", hdu=2))

2023

In [75]:
class PSFex(readWriteCats):
    def __init__(self, catalog=None, command = "psfex", folder = "./outputs/"):
        readWriteCats.__init__(self)

        self.command = command
        self.catalog = catalog
        self.folder = folder

        self.config = {
            #PSF model
            'BASIS_TYPE': {'value': 'PIXEL_AUTO', 'comment': 'NONE, PIXEL, GAUSS-LAGUERRE or FILE'},
            'BASIS_NUMBER': {'value': '20', 'comment': 'Basis number or parameter'},
            'BASIS_NAME': {'value': 'basis.fits', 'comment': 'Basis filename (FITS data-cube)'},
            'BASIS_SCALE': {'value': '1.0', 'comment': 'Gauss-Laguerre beta parameter'},
            'NEWBASIS_TYPE': {'value': 'NONE', 'comment': ' or PCA_COMMON'},
            'NEWBASIS_NUMBER': {'value': '8', 'comment': 'Number of new basis vectors'},
            'PSF_SAMPLING': {'value': '0.0', 'comment': 'Sampling step in pixel units (0.0 = auto)'},
            'PSF_PIXELSIZE': {'value': '1.0', 'comment': 'Effective pixel size in pixel step units'},
            'PSF_ACCURACY': {'value': '0.01', 'comment': 'Accuracy to expect from PSF "pixel" values'},
            'PSF_SIZE': {'value': '25,25', 'comment': 'Image size of the PSF model'},
            'PSF_DGEOCORRECT': {'value': 'N', 'comment': 'Use diff. geom. maps (if provided) Y/N?'},
            'PSF_RECENTER': {'value': 'N', 'comment': 'Allow recentering of PSF-candidates Y/N ?'},
            'MEF_TYPE': {'value': 'INDEPENDENT', 'comment': 'INDEPENDENT or COMMON'},

            #Point source measurements
            'CENTER_KEYS': {'value': 'X_IMAGE,Y_IMAGE', 'comment': 'Catalogue parameters for source pre-centering'},
            'PHOTFLUX_KEY': {'value': 'FLUX_APER(1)', 'comment': 'Catalogue parameter for photometric norm.'},
            'PHOTFLUXERR_KEY': {'value': 'FLUXERR_APER(1)', 'comment': 'Catalogue parameter for photometric error'},
            
            #PSF variability
            'PSFVAR_KEYS': {'value': 'X_IMAGE,Y_IMAGE', 'comment': 'Catalogue or FITS (preceded by :) params'},
            'PSFVAR_GROUPS': {'value': '1,1', 'comment': 'Group tag for each context key'},
            'PSFVAR_DEGREES': {'value': '2', 'comment': 'Polynom degree for each group'},
            'PSFVAR_NSNAP': {'value': '9', 'comment': 'Number of PSF snapshots per axis'},
            'HIDDENMEF_TYPE': {'value': 'COMMON', 'comment': 'INDEPENDENT or COMMON'},
            'STABILITY_TYPE': {'value': 'EXPOSURE', 'comment': 'EXPOSURE or SEQUENCE'},
            
            #Sample selection
            'SAMPLE_AUTOSELECT': {'value': 'Y', 'comment': 'Automatically select the FWHM (Y/N) ?'},
            'SAMPLEVAR_TYPE': {'value': 'SEEING', 'comment': 'File-to-file PSF variability: NONE or SEEING'},
            'SAMPLE_FWHMRANGE': {'value': '2.0,10.0', 'comment': 'Allowed FWHM range'},
            'SAMPLE_VARIABILITY': {'value': '0.2', 'comment': 'Allowed FWHM variability (1.0 = 100%)'},
            'SAMPLE_MINSN': {'value': '20', 'comment': 'Minimum S/N for a source to be used'},
            'SAMPLE_MAXELLIP': {'value': '0.3', 'comment': 'Maximum (A-B)/(A+B) for a source to be used'},
            'SAMPLE_FLAGMASK': {'value': '0x00fe', 'comment': 'Rejection mask on SExtractor FLAGS'},
            'SAMPLE_WFLAGMASK': {'value': '0x0000', 'comment': 'Rejection mask on SExtractor FLAGS_WEIGHT'},
            'SAMPLE_IMAFLAGMASK': {'value': '0x0', 'comment': 'Rejection mask on SExtractor IMAFLAGS_ISO'},
            'BADPIXEL_FILTER': {'value': 'N', 'comment': 'Filter bad-pixels in samples (Y/N) ?'},
            'BADPIXEL_NMAX': {'value': '0', 'comment': 'Maximum number of bad pixels allowed'},
            
            #PSF homogeneisation kernel
            'HOMOBASIS_TYPE': {'value': 'NONE', 'comment': 'NONE or GAUSS-LAGUERRE'},
            'HOMOBASIS_NUMBER': {'value': '10', 'comment': 'Kernel basis number or parameter'},
            'HOMOBASIS_SCALE': {'value': '1.0', 'comment': 'GAUSS-LAGUERRE beta parameter'},
            'HOMOPSF_PARAMS': {'value': '2.0, 3.0', 'comment': 'Moffat parameters of the idealised PSF'},
            'HOMOKERNEL_DIR': {'value': '', 'comment': 'Where to write kernels (empty=same as input)'},
            'HOMOKERNEL_SUFFIX': {'value': '.homo.fits', 'comment': 'Filename extension for homogenisation kernels'},
            
            #Output catalogs
            'OUTCAT_TYPE': {'value': 'NONE', 'comment': 'NONE, ASCII_HEAD, ASCII, FITS_LDAC'},
            'OUTCAT_NAME': {'value': 'psfex_out.cat', 'comment': 'Output catalog filename'},
            
            #Check-plots
            'CHECKPLOT_DEV': {'value': 'PNG', 'comment': ' JPEG, AQT, PDF or SVG'},
            'CHECKPLOT_RES': {'value': '0', 'comment': 'Check-plot resolution (0 = default)'},
            'CHECKPLOT_ANTIALIAS': {'value': 'Y', 'comment': 'Anti-aliasing using convert (Y/N) ?'},
            'CHECKPLOT_TYPE': {'value': 'NONE', 'comment': 'SELECTION_FWHM,FWHM,ELLIPTICITY,COUNTS,COUNT_FRACTION,CHI2, RESIDUALS or NONE'},
            'CHECKPLOT_NAME': {'value': 'selfwhm,fwhm,ellipticity,counts,countfrac,chi2,resi', 'comment': ''},
            
            #Check-Images
            'CHECKIMAGE_TYPE': {'value': 'NONE', 'comment': 'CHI,PROTOTYPES,SAMPLES,RESIDUALS,SNAPSHOTS or MOFFAT,-MOFFAT,-SYMMETRICAL'},
            'CHECKIMAGE_NAME': {'value': 'chi.fits,proto.fits,samp.fits,resi.fits,snap.fits', 'comment': ' Check-image filenames'},
            'CHECKIMAGE_CUBE': {'value': 'N', 'comment': 'Save check-images as datacubes (Y/N) ?'},
            'PSF_DIR': {'value': os.path.join(folder, ''), 'comment': 'Where to write PSFs (empty=same as input)'},
            'PSF_SUFFIX': {'value': '.psf', 'comment': 'Filename extension for output PSF filename'},
            'VERBOSE_TYPE': {'value': 'NORMAL', 'comment': 'can be QUIET,NORMAL,LOG or FULL'},
            'WRITE_XML': {'value': 'Y', 'comment': 'Write XML file (Y/N)?'},
            'XML_NAME': {'value': os.path.join(folder, 'psfex.xml'), 'comment': 'Filename for XML output'},
            'XSL_URL': {'value': 'file:///usr/local/share/psfex/psfex.xsl', 'comment': ' Filename for XSL style-sheet'},
            'NTHREADS': {'value': '0', 'comment': ' 0 = automatic'}

        }

    def run(self):
        self.confile = os.path.join(self.folder, 'config.psfex')
        self.write_file(self.confile, mode="scamp")

        MarManager.INFO('Running - PSFex')

        command = self.command + " " + self.catalog + " -c " + self.confile

        os.system(command)

    def getPSFinfo(self):
        v = votable.parse(os.path.join(self.folder, 'psfex.xml'))
        table = v.get_table_by_id('PSF_Fields')

        data = table.array

        keys = ["NStars_Loaded_Total",
                "NStars_Accepted_Total",
                "FWHM_Mean",  # pixel unit
                "FWHM_Min",  # pixel
                "FWHM_Max",  # pixel 
                "Chi2_Mean",
                "MoffatBeta_Mean",
                "Asymmetry_Mean",
                "Ellipticity_Mean"]

        # return {key: data[key].data[0] for key in keys}
        info = {}
        for key in keys:
            info[key] = data[key].data[0]
        
        return info

In [76]:
psf = PSFex(catalog="t.fits")

In [77]:
psf.run()


> WARNING: This executable has been compiled using a version of the ATLAS library without support for multithreading. Performance will be degraded.

> 
----- PSFEx 3.23.0 started on 2023-02-07 at 14:41:18 with 12 threads

> 
----- 1 input catalogues:
t.fits              :  "STRIPE82_0002   "    1 extension     324 detections

> Initializing contexts...
> Computing optimum PSF sampling steps...
> Reading data from t...
> Computing final PSF model for t...
   filename      [ext] accepted/total samp. chi2/dof FWHM ellip. resi. asym.
> Computing diagnostics for t...
t                          235/317     0.82   1.35   2.92  0.02  0.02  0.01
> Saving PSF model and metadata for t...
> Writing XML file...
> 
> All done (in 0.0 s)


In [78]:
psf.getPSFinfo()

{'NStars_Loaded_Total': 317,
 'NStars_Accepted_Total': 235,
 'FWHM_Mean': 2.91726,
 'FWHM_Min': 2.82655,
 'FWHM_Max': 3.17903,
 'Chi2_Mean': 1.35199,
 'MoffatBeta_Mean': 2.51649,
 'Asymmetry_Mean': 0.0143011,
 'Ellipticity_Mean': 0.0154316}

In [7]:
import mar

In [9]:
mar.wrappers.createVignetCatalog("/home/gustavo/Downloads/_media_files_HYDRA-0111_F378_swp", "./outputs/final.test.fits")

> 
----- SExtractor 2.25.2 started on 2023-02-07 at 17:06:24 with 1 thread

> Setting catalog parameters
> Reading detection filter
> Initializing Neural Network
> Reading Neural Network Weights
> Initializing catalog
> Looking for _media_files_HYDRA-0111_F378_swp
----- Measuring from: _media_files_HYDRA-0111_F378_swp
      "HYDRA_0111" / no ext. header / 11000x11000 / 32 bits (floats)
Detection+Measurement image: > Setting up background maps
> Setting up background map at line:  512
> Setting up background map at line: 1024
> Setting up background map at line: 1536
> Setting up background map at line: 2048
> Setting up background map at line: 2560
> Setting up background map at line: 3072
> Setting up background map at line: 3584
> Setting up background map at line: 4096
> Setting up background map at line: 4608
> Setting up background map at line: 5120
> Setting up background map at line: 5632
> Setting up background map at line: 6144
> Setting up background map at line: 6656
> Setti

In [10]:
psf = mar.wrappers.PSFex("outputs/final.test.fits")


psf.run()


> WARNING: This executable has been compiled using a version of the ATLAS library without support for multithreading. Performance will be degraded.

> 
----- PSFEx 3.23.0 started on 2023-02-07 at 17:06:29 with 12 threads

> 
----- 1 input catalogues:
final.test.fits     :  "HYDRA_0111      "    1 extension     861 detections

> Initializing contexts...
> Computing optimum PSF sampling steps...
> Reading data from final.test...
> Computing final PSF model for final.test...
   filename      [ext] accepted/total samp. chi2/dof FWHM ellip. resi. asym.
> Computing diagnostics for final.test...
final.test                 659/831     0.70   1.87   2.44  0.01  0.02  0.02
> Saving PSF model and metadata for final.test...
> Writing XML file...
> 
> All done (in 0.0 s)


In [11]:
psf.getPSFinfo()

{'NStars_Loaded_Total': 831,
 'NStars_Accepted_Total': 659,
 'FWHM_Mean': 2.4359,
 'FWHM_Min': 2.31162,
 'FWHM_Max': 2.67902,
 'Chi2_Mean': 1.87127,
 'MoffatBeta_Mean': 2.57804,
 'Asymmetry_Mean': 0.0151009,
 'Ellipticity_Mean': 0.0147957}